# Appliead Data Science Capstone

#### Stefan Ubovic

### Peer-graded assignment: Scraping postcodes from Wikipedia

In [105]:
import pandas as pd
import numpy as np

Download and read HTML using urllib:

In [107]:
import urllib.request, urllib.parse, urllib.error
import xml.etree.ElementTree as ET
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fhand = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

html = fhand.read().decode()

Parse HTML using BeautifulSoup:

In [108]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

cells = soup('td') #'td' denotes table values within the soup object

# creates dataframe with required number of rows and columns
rows = int(len(cells)/3) # number of rows
neighbourhoods_raw = pd.DataFrame(index=np.arange(rows), columns=['Post Code', 'Borough', 'Neighbourhood'])

# fills in dataframe with relevant values from soup object
row = 0
column = 0

for cell in cells:
    neighbourhoods_raw.iloc[row,column] = cell.text.rstrip()
    column = column + 1
    
    if column==3:
        column = 0
        row = row + 1

neighbourhoods_raw

,Post Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
186,E,G,H
187,J,K,L
188,M,N,P
189,R,S,T


Wrangle dataframe: remove invalid values and unassigned boroughs

In [109]:
neighbourhoods = neighbourhoods_raw.drop(range(180,191)) # removes rows containing values that are not post codes
neighbourhoods = neighbourhoods[(neighbourhoods['Borough']!='Not assigned')] # drops post codes where borough not assigned
neighbourhoods

,Post Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Check if any neighbourhoods are still unassigned:

In [110]:
if 'Not assigned' in neighbourhoods.Neighbourhood.values:
    print('Further wrangling required.')
else: 
    print("There are no 'Not assigned' values.")

# print(neighbourhoods.Neighbourhood.values)

There are no 'Not assigned' values.


In [111]:
print('There are {} assigned postcodes in Toronto.'.format(neighbourhoods.shape[0]))

There are 103 assigned postcodes in Toronto.
